## Note: 
- This code is to perform image processing including registration and N4 bias correction via Nipype, skull-stripping, automated tumor segmentation, extraction of tumor loci and shape features, normalization and IDH status prediction which is the final step of hybrid model. 

<img src="workflow.png">
  
- Python module requirements  : Nipype /  FSL /  ANTs / PyRadiomics / PyTorch
- The process resquires GPU.

#### Use this to load a dumped session. (See last cell to dump session)

In [ ]:

#import dill
#dill.load_session('notebook_env.db')

### Image processing - skullstripping, resampling, registration and bias correction

In [ ]:
from img_processing import *

In [ ]:
    # file path of original T1C 
T1C_original_file = 'T1C.nii.gz'            
T2_original_file = 'T2.nii.gz'
FLAIR_original_file = 'FLAIR.nii.gz'
    
    # file path of skull-stripped images
T1C_bet_file = 't1c_bet.nii.gz'            
T2_bet_file = 't2_bet.nii.gz'
FLAIR_bet_file = 'flair_bet.nii.gz'
    
    # file path of the mask for T1C skull stripping
brainmask_T1C_file = 'T1C_bet0_temp_mask.nii.gz'  

# filenames to save isovoxel images / brain mask
T1C_iso_file = 't1c_isovoxel.nii.gz'
T2_iso_file = 't2_isovoxel.nii.gz'
FLAIR_iso_file = 'flair_isovoxel.nii.gz'
brainmask_iso_file = 'mask_brain_isovoxel.nii.gz'    

# filenames to save bias-corrected images   
T1C_corrected_file = 't1c_corrected.nii.gz'
T2_corrected_file = 't2_corrected.nii.gz'
FLAIR_corrected_file = 'flair_corrected.nii.gz'

## filename for preliminary skull-stripped T1C                                       
T1C_bet_temp_file = 'T1C_bet0_temp.nii.gz'

In [ ]:
func_img_proc(T1C_original_file, T2_original_file, FLAIR_original_file,
              T1C_bet_file, T2_bet_file, FLAIR_bet_file, brainmask_T1C_file,
              T1C_iso_file, T2_iso_file, FLAIR_iso_file, brainmask_iso_file,
              T1C_corrected_file, T2_corrected_file, FLAIR_corrected_file,
              T1C_bet_temp_file)

### Model 1 : Automatic tumor segmentation  


#### Preprocessing for Model 1

In [ ]:
(t1c_unet_arr, flair_unet_arr, cropdown_info) = func_norm_model1(T1C_corrected_file, FLAIR_corrected_file, brainmask_iso_file)

# cropdown_info will be used for resmampling the predicted tumor mask to original isovoxel space, and preprcessing for Model2.
cropdown_info

#### Get tumor mask from Model 1

In [ ]:

predmask_arr = func_get_predmask(t1c_unet_arr, flair_unet_arr)

#### Resample the predicted mask back to original isovoxel space

In [ ]:

predmask_isovoxel_arr = func_mask_back2iso(predmask_arr, cropdown_info)
predmask_isovoxel_arr_sitk = np.transpose(predmask_isovoxel_arr, (2,1,0))
predmask_isovoxel_img = sitk.GetImageFromArray(predmask_isovoxel_arr_sitk)

predmask_isovoxel_file = 'predmask_isovoxel.nii.gz' #filename for predicted mask of isovoxel resolution
sitk.WriteImage(predmask_isovoxel_img, predmask_isovoxel_file)   # save the automatic segmentation of isovoxel resolution

### Model 2 : CNN classifier for IDH status prediction

#### Preprocessing for Model 2

In [ ]:
t1c_corrected_img = nb.load(T1C_corrected_file)
t1c_corrected_arr = t1c_corrected_img.get_data()
t2_corrected_img = nb.load(T2_corrected_file)
t2_corrected_arr = t2_corrected_img.get_data()
brain_mask = nb.load(brainmask_iso_file)
brain_mask_arr = brain_mask.get_data()

t1c_resnet_arr = func_norm_resnet(t1c_corrected_arr, predmask_isovoxel_arr, brain_mask_arr, cropdown_info)
t2_resnet_arr = func_norm_resnet(t2_corrected_arr, predmask_isovoxel_arr, brain_mask_arr, cropdown_info)

    

#### Get shape and loci features from tumor mask of 1mm isovoxel

In [ ]:
sla_features = func_shapeloci(T1C_iso_file, predmask_isovoxel_file)

#### Add patient's age

In [ ]:
sla_features['age'] = pd.Series(50)  ## change 50 to the patient's age.

In [ ]:
#df = sla_features

#### Normalize features

In [ ]:
sla_features_mean = pd.read_csv('sla_features_mean.csv')
sla_features_norm = sla_features

In [ ]:
#x-mean
for i in range(len(sla_features.columns)):
    sla_features_norm.iloc[0, i] = sla_features.iloc[0, i] - sla_features_mean.iloc[0, i]

In [ ]:
#show normalized features
sla_features_norm

####

In [ ]:
sla_features_norm = np.array(sla_features)
sla_arr = np.repeat(sla_features_norm, 5, axis=0)
sla_arr.shape

#### Get probability of IDH mutation from Model 2

In [ ]:
#download Model 2
from torch.hub import download_url_to_file

url='https://github.com/yoonchoi-neuro/automated_hybrid_IDH/releases/download/final/MODEL2_CNNclassifier.pth'
download_url_to_file(url, 'MODEL2_CNNclassifier.pth')

In [ ]:
output_mean = get_IDH_pred(t1c_resnet_arr, t2_resnet_arr, predmask_arr, sla_arr)

In [ ]:
#Dump current session so you can reload it later without needing to run everything again. func_img_proc() takes quite a lot of time. It might be wise to run this cell immediately after that function ends.
import dill
dill.dump_session('notebook_env.db')